# 목적

바로 VGG19를 넣으려고 했지만, 학습 시간 및 효율 문제로 인해 의도한 결과가 나오지 않았다.

프로젝트 개발 방향과 순서를 조금 바꾼다.

우선 완전 간단한 CNN 모델을 만들어서 빠르게 학습시키고, 어느정도 정확도가 보장된 모델 (>80%)이 완성되면 해당 모델을 활용한 웹 인터페이스를 제작한다.

이후 적당히 싼 다른 구조를 빌려오거나, VGG19 pre-trained 파라미터를 빌려와서 사용하거나, 생으로 VGG19를 돌리거나 해서 좀 더 높은 accuracy 모델이 생성되면 모델을 교체하는 방식으로 발전시키자.

# 0.1. 구글 드라이브 연동

In [1]:
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# 0.2. 라이브러리 임포트 및 전역변수 설정

In [2]:
from pathlib import Path
from easydict import EasyDict as edict
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset, random_split
from collections import Counter
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
root = '/gdrive/My Drive/ctp431'
inst_pool = ['cel', 'cla', 'flu', 'gac', 'gel', 'org', 'pia', 'sax', 'tru', 'vio', 'voi']

# Step 1. 간단한 CNN 모델 만들기

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self, num_class = 11):
        super(SimpleCNN, self).__init__()

        self.conv_block = nn.Sequential(
            # block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_block = nn.Sequential(
            # block 6
            nn.Flatten(),
            nn.Linear(128 * 56 * 56, 200),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(200, num_class)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc_block(x)
        return x

# Step 2. 학습 돌리기

In [5]:
# 데이터 로드
test_data_path = root + "/MK2/DataSet/test_data.pt"
test_x, test_y = torch.load(test_data_path)

train_data_path = root + "/MK2/DataSet/training_data.pt"
train_x, train_y = torch.load(train_data_path)

# 데이터셋과 DataLoader 생성
train_data = TensorDataset(train_x, train_y)
test_data = TensorDataset(test_x, test_y)

train_loader = DataLoader(train_data, batch_size=35, shuffle=True)
val_loader = DataLoader(test_data, batch_size=35, shuffle=False)

<ipython-input-5-d92628b8d171>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_x, test_y = torch.load(test_data_path)
<ipython-input-5-d92628b8d171>:6: FutureWarning: 

In [6]:
# 모델, 손실 함수, 옵티마이저 초기화
model = SimpleCNN(num_class=11)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def initialize_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

model.apply(initialize_weights)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

result_dir = Path(root) / 'MK2/Results/Simple_CNN'
result_dir.mkdir(parents=True, exist_ok=True)

# Create directory name.
num_trial=0
parent_dir = result_dir / f'trial_{num_trial}'
while parent_dir.is_dir():
    num_trial = int(parent_dir.name.replace('trial_',''))
    parent_dir = result_dir / f'trial_{num_trial+1}'
parent_dir.mkdir(parents=True, exist_ok=True)
print(f'Trial save path : {parent_dir}')

# 트레이닝 루프
epochs = 10
for epoch in tqdm(range(epochs)):
    print("\n")
    model.train()
    running_loss = 0.0
    train_count = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_count += inputs.size(0)
        print(f"Epoch {epoch}/ train_count = {train_count}")

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss / len(val_loader):.4f}, Accuracy: {val_accuracy:.2f}%")


model_save_path = str(parent_dir) + '/simple_cnn_model.pt'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Trial save path : /gdrive/My Drive/ctp431/MK2/Results/Simple_CNN/trial_3


  0%|          | 0/10 [00:00<?, ?it/s]



Epoch 0/ train_count = 35
Epoch 0/ train_count = 70
Epoch 0/ train_count = 105
Epoch 0/ train_count = 140
Epoch 0/ train_count = 175
Epoch 0/ train_count = 210
Epoch 0/ train_count = 245
Epoch 0/ train_count = 280
Epoch 0/ train_count = 315
Epoch 0/ train_count = 350
Epoch 0/ train_count = 385
Epoch 0/ train_count = 420
Epoch 0/ train_count = 455
Epoch 0/ train_count = 490
Epoch 0/ train_count = 525
Epoch 0/ train_count = 560
Epoch 0/ train_count = 595
Epoch 0/ train_count = 630
Epoch 0/ train_count = 665
Epoch 0/ train_count = 700
Epoch 0/ train_count = 735
Epoch 0/ train_count = 770
Epoch 0/ train_count = 805
Epoch 0/ train_count = 840
Epoch 0/ train_count = 875
Epoch 0/ train_count = 910
Epoch 0/ train_count = 945
Epoch 0/ train_count = 980
Epoch 0/ train_count = 1015
Epoch 0/ train_count = 1050
Epoch 0/ train_count = 1085
Epoch 0/ train_count = 1120
Epoch 0/ train_count = 1155
Epoch 0/ train_count = 1190
Epoch 0/ train_count = 1225
Epoch 0/ train_count = 1260
Epoch 0/ train_count

 10%|█         | 1/10 [00:28<04:19, 28.84s/it]

Validation Loss: 2.3296, Accuracy: 16.23%


Epoch 1/ train_count = 35
Epoch 1/ train_count = 70
Epoch 1/ train_count = 105
Epoch 1/ train_count = 140
Epoch 1/ train_count = 175
Epoch 1/ train_count = 210
Epoch 1/ train_count = 245
Epoch 1/ train_count = 280
Epoch 1/ train_count = 315
Epoch 1/ train_count = 350
Epoch 1/ train_count = 385
Epoch 1/ train_count = 420
Epoch 1/ train_count = 455
Epoch 1/ train_count = 490
Epoch 1/ train_count = 525
Epoch 1/ train_count = 560
Epoch 1/ train_count = 595
Epoch 1/ train_count = 630
Epoch 1/ train_count = 665
Epoch 1/ train_count = 700
Epoch 1/ train_count = 735
Epoch 1/ train_count = 770
Epoch 1/ train_count = 805
Epoch 1/ train_count = 840
Epoch 1/ train_count = 875
Epoch 1/ train_count = 910
Epoch 1/ train_count = 945
Epoch 1/ train_count = 980
Epoch 1/ train_count = 1015
Epoch 1/ train_count = 1050
Epoch 1/ train_count = 1085
Epoch 1/ train_count = 1120
Epoch 1/ train_count = 1155
Epoch 1/ train_count = 1190
Epoch 1/ train_count = 1225
Epoch 

...

 90%|█████████ | 9/10 [04:14<00:28, 28.35s/it]

Validation Loss: 3.0513, Accuracy: 29.87%


Epoch 9/ train_count = 35
Epoch 9/ train_count = 70
Epoch 9/ train_count = 105
Epoch 9/ train_count = 140
Epoch 9/ train_count = 175
Epoch 9/ train_count = 210
Epoch 9/ train_count = 245
Epoch 9/ train_count = 280
Epoch 9/ train_count = 315
Epoch 9/ train_count = 350
Epoch 9/ train_count = 385
Epoch 9/ train_count = 420
Epoch 9/ train_count = 455
Epoch 9/ train_count = 490
Epoch 9/ train_count = 525
Epoch 9/ train_count = 560
Epoch 9/ train_count = 595
Epoch 9/ train_count = 630
Epoch 9/ train_count = 665
Epoch 9/ train_count = 700
Epoch 9/ train_count = 735
Epoch 9/ train_count = 770
Epoch 9/ train_count = 805
Epoch 9/ train_count = 840
Epoch 9/ train_count = 875
Epoch 9/ train_count = 910
Epoch 9/ train_count = 945
Epoch 9/ train_count = 980
Epoch 9/ train_count = 1015
Epoch 9/ train_count = 1050
Epoch 9/ train_count = 1085
Epoch 9/ train_count = 1120
Epoch 9/ train_count = 1155
Epoch 9/ train_count = 1190
Epoch 9/ train_count = 1225
Epoch 

100%|██████████| 10/10 [04:43<00:00, 28.33s/it]

Validation Loss: 2.8215, Accuracy: 26.49%


Model saved to /gdrive/My Drive/ctp431/MK2/Results/Simple_CNN/trial_3/simple_cnn_model.pt
